In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.gridspec import GridSpec
from scipy.ndimage import gaussian_filter
from tkinter.simpledialog import askfloat
from tkinter import simpledialog
from sklearn.cluster import MeanShift, estimate_bandwidth

In [11]:
class ToolTip:
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tooltip = None
        self.widget.bind("<Enter>", self.enter)
        self.widget.bind("<Leave>", self.leave)
        
    def enter(self, event=None):
        x, y, _, _ = self.widget.bbox("insert")
        x += self.widget.winfo_rootx() + 25
        y += self.widget.winfo_rooty() + 25
        self.tooltip = tk.Toplevel(self.widget)
        self.tooltip.wm_overrideredirect(True)
        self.tooltip.wm_geometry(f"+{x}+{y}")
        label = tk.Label(self.tooltip, text=self.text, 
                         background="#2b2b2b", foreground="white", relief="solid",
                         borderwidth=0, padx=10, pady=5)
        label.pack()
        
    def leave(self, event=None):
        if self.tooltip:
            self.tooltip.destroy()

In [12]:
class ImageProcessorApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Image Enhancement/Segmentation project")
        self.master.configure(background='#2b2b2b')
        self.master.iconbitmap('image processing icon.ico')

        self.image_frame = tk.Frame(self.master, background='#2b2b2b')
        self.image_frame.pack()

        self.button_frame = tk.Frame(self.master, background='#2b2b2b')
        self.button_frame.pack()

        self.open_image_btn = tk.Button(self.button_frame, text="Open Image", command=self.open_image, bg="#4169E1", fg="white", activebackground="#6495ED", activeforeground="white", relief="raised")
        self.open_image_btn.grid(row=0, column=0, padx=5, pady=5)
        ToolTip(self.open_image_btn, "Open an image file\n*.png;*.jpg;*.jpeg")

        self.enhancement1_btn = tk.Button(self.button_frame, text="Histogram Enhancement", command=self.Histogram_Enhancement, bg="#008B8B", fg="white", activebackground="#20B2AA", activeforeground="white", relief="raised")
        self.enhancement1_btn.grid(row=0, column=1, padx=5, pady=5)
        ToolTip(self.enhancement1_btn, "**Histogram enhancement**\n Adjusts the contrast and brightness of an image by redistributing pixel intensity values")

        self.enhancement2_btn = tk.Button(self.button_frame, text="Gaussian Filter", command=self.Gaussian_Filter, bg="#FF4500", fg="white", activebackground="#FF6347", activeforeground="white", relief="raised")
        self.enhancement2_btn.grid(row=0, column=2, padx=5, pady=5)
        ToolTip(self.enhancement2_btn, "**Gaussian filter**\n- Smooths an image by convolving it with a Gaussian kernel\n- reducing noise and preserving edges.")

        self.segmentation1_btn = tk.Button(self.button_frame, text="Mean shift Segmentation", command=self.Mean_shift_Segmentation, bg="#6A5ACD", fg="white", activebackground="#9370DB", activeforeground="white", relief="raised")
        self.segmentation1_btn.grid(row=0, column=3, padx=5, pady=5)
        ToolTip(self.segmentation1_btn, "**Mean shift segmentation**\n - Iteratively shifts data points towards the mode of their local density distribution\n - grouping similar pixels into clusters based on their feature similarity")
        
    
        self.segmentation2_btn = tk.Button(self.button_frame, text="Gaussian Adaptive Thresholding", command=self.gaussian_threshold, bg="#228B22", fg="white", activebackground="#32CD32", activeforeground="white", relief="raised")
        self.segmentation2_btn.grid(row=0, column=4, padx=5, pady=5)
        ToolTip(self.segmentation2_btn, "**Gaussian adaptive thresholding**\nSegments an image by thresholding each pixel based on the local Gaussian-weighted mean of its neighborhood\n- adapting to varying lighting conditions.")
        
        self.clear_canvas_btn = tk.Button(self.button_frame, text="Remove all effects", command=self.clear_canvas, bg="#FFA500", fg="#2b2b2b", activebackground="#FF8C00", activeforeground="white", relief="raised")
        self.clear_canvas_btn.grid(row=0, column=5, padx=5, pady=5)
        ToolTip(self.clear_canvas_btn, "Remove all effects and reset the image")
                                                                                                                           #test white                                                                                                  
        self.download_btn = tk.Button(self.button_frame, text="Download Image", command=self.download_image, bg="#FFD700", fg="#2b2b2b", activebackground="#FFD700", activeforeground="white", relief="raised")
        self.download_btn.grid(row=0, column=6, padx=5, pady=5)
        ToolTip(self.download_btn, "- Download the modified image as png to your local machine")
        
        self.image_label = tk.Label(self.image_frame)
        self.image_label.pack()

        self.current_image = None
        self.modified_image = None
        self.canvas = None

    def open_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            self.current_image = Image.open(file_path)
            self.display_image()

    def display_image(self):
        if self.current_image:
            self.current_image.thumbnail((400, 400))
            photo = ImageTk.PhotoImage(self.current_image)
            self.image_label.config(image=photo)
            self.image_label.image = photo

    def Histogram_Enhancement(self):  #equ hist
        if self.current_image:
            img = cv2.cvtColor(np.array(self.current_image), cv2.COLOR_RGB2GRAY)
            imaga_equ = cv2.equalizeHist(img)

            fig = Figure(figsize=(10, 6))
            gs = GridSpec(2, 2, figure=fig)

            ax1 = fig.add_subplot(gs[0, 0])
            ax2 = fig.add_subplot(gs[0, 1])
            ax3 = fig.add_subplot(gs[1, 0])
            ax4 = fig.add_subplot(gs[1, 1])

            ax1.imshow(self.current_image)
            ax1.set_title('Original Image')

            ax2.imshow(imaga_equ, cmap='gray')
            ax2.set_title('Enhanced Image')

            ax3.hist(img.ravel(), bins=256, range=(0, 256), color='black')
            ax3.set_title('Histogram Before Enhancement')
            ax3.set_xlabel('Pixel Intensity')
            ax3.set_ylabel('Frequency')

            ax4.hist(imaga_equ.ravel(), bins=256, range=(0, 256), color='black')
            ax4.set_title('Histogram After Enhancement')
            ax4.set_xlabel('Pixel Intensity')
            ax4.set_ylabel('Frequency')

            fig.subplots_adjust(wspace=0.5, hspace=0.5)

            if self.canvas:
                self.canvas.get_tk_widget().destroy()

            self.canvas = FigureCanvasTkAgg(fig, master=self.master)
            self.canvas.draw()
            self.canvas.get_tk_widget().pack()

            self.modified_image = Image.fromarray(imaga_equ)
            self.display_modified_image()

    def Gaussian_Filter(self):
        if self.current_image:

            mean = askfloat("Input", "Enter the mean:", parent=self.master)
            if mean is None: 
                return
            stdev = askfloat("Input", "Enter the standard deviation:", parent=self.master)
            if stdev is None:
                return

            image = cv2.cvtColor(np.array(self.current_image), cv2.COLOR_RGB2GRAY)
            filteredImg = gaussian_filter(image, stdev, order=0, mode='constant', cval=mean)

            fig = Figure(figsize=(12, 4))
            gs = GridSpec(1, 2, figure=fig)

            ax1 = fig.add_subplot(gs[0, 0])
            ax1.set_title('Original Image')
            ax1.imshow(image, cmap='gray')
            ax1.axis('off')

            ax2 = fig.add_subplot(gs[0, 1])
            ax2.set_title('Gaussian filter')
            ax2.imshow(filteredImg, cmap='gray')
            ax2.axis('off')

            if self.canvas:
                self.canvas.get_tk_widget().destroy()

            self.canvas = FigureCanvasTkAgg(fig, master=self.master)
            self.canvas.draw()
            self.canvas.get_tk_widget().pack()

            self.modified_image = Image.fromarray(filteredImg)
            self.display_modified_image()


    def Mean_shift_Segmentation(self):
        if self.current_image:

            originImg = np.array(self.current_image)  

            originShape = originImg.shape

            flatImg = np.reshape(originImg, [-1, 3])


            bandwidth = estimate_bandwidth(flatImg, quantile=0.1, n_samples=100)
            ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)

            ms.fit(flatImg)

            labels = ms.labels_
            cluster_centers = ms.cluster_centers_

            segmentedImg = cluster_centers[np.reshape(labels, originShape[:2])]

            fig = Figure(figsize=(12, 4))
            gs = GridSpec(1, 2, figure=fig)

            ax1 = fig.add_subplot(gs[0, 0])
            ax1.set_title('Original Image')
            ax1.imshow(originImg, cmap='gray')
            ax1.axis('off')

            ax2 = fig.add_subplot(gs[0, 1])
            ax2.set_title('Mean shift segmentation')
            ax2.imshow(segmentedImg.astype(np.uint8))  
            ax2.axis('off')

            if self.canvas:
                self.canvas.get_tk_widget().destroy()

            self.canvas = FigureCanvasTkAgg(fig, master=self.master)
            self.canvas.draw()
            self.canvas.get_tk_widget().pack()

            self.modified_image = Image.fromarray(segmentedImg.astype(np.uint8))
            self.display_modified_image()

    def gaussian_threshold(self):      
            img = cv2.cvtColor(np.array(self.current_image), cv2.COLOR_RGB2GRAY)
            thresholded_image = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

            fig = Figure(figsize=(12, 4))
            gs = GridSpec(1, 2, figure=fig)

            ax1 = fig.add_subplot(gs[0, 0])
            ax1.set_title('Original Image')
            ax1.imshow(img, cmap='gray')
            ax1.axis('off')

            ax2 = fig.add_subplot(gs[0, 1])
            ax2.set_title('Gaussian adaptive thresholding')
            ax2.imshow(thresholded_image, cmap='gray')
            ax2.axis('off')

            if self.canvas:
                self.canvas.get_tk_widget().destroy()

            self.canvas = FigureCanvasTkAgg(fig, master=self.master)
            self.canvas.draw()
            self.canvas.get_tk_widget().pack()

            self.modified_image = Image.fromarray(thresholded_image)
            self.display_modified_image()

    def display_modified_image(self):
        if self.modified_image:
            modified_photo = ImageTk.PhotoImage(self.modified_image)
            self.image_label.config(image=modified_photo)
            self.image_label.image = modified_photo
    
    def clear_canvas(self):
        self.current_image = self.current_image
        self.modified_image = None
        self.display_image()
        if self.canvas:
            self.canvas.get_tk_widget().destroy()

    def download_image(self):
        if self.modified_image:
            file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
            if file_path:
                self.modified_image.save(file_path)

In [13]:
def main():
    root = tk.Tk()
    app = ImageProcessorApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()